In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("VS17MORT.csv") # Mortality dataset's DATAFRAME

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head(index=False)

TypeError: head() got an unexpected keyword argument 'index'